### Prepare fcs files for deep learning
This is a small example for formatting data from fcs files into numpy array, and save the metaData, marker names and the numpy array into allData.obj file. Use the script as a template to prepare your own fcs files for deep learning. 

In [2]:
import sys
!{sys.executable} -m pip install rpy2

     |████████████████████████████████| 218 kB 1.6 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for rpy2: filename=rpy2-3.5.13-cp39-cp39-linux_x86_64.whl size=262982 sha256=5e6e25c7e33c498ba4493aeea912f6e91f6ad246e344401572025d9298f7fc74
  Stored in directory: /home/ewillie/.cache/pip/wheels/6b/78/1f/91040f2bc8e2ae1f542ec592ec60f2d57087c33f6e4889e67f
Successfully built rpy2


In [15]:
import numpy as np
import scipy as sp
import pandas as pd
import rpy2 as rp
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter
import os 
import rpy2.robjects as ro
import pickle
from collections import Counter


# import R's "flowCore" package
utils = importr('flowCore')
os.chdir('/home/ewillie/Documents/PhD/DeepLearning_CyTOF/COVID_19_PBMC_Mathew_2020/data/data_sub_10k')

In [16]:
##### list fcs files #####
cytof_files = pd.read_csv("../Covid_Bcell_metadata.csv")
print(cytof_files)
fn = [os.path.join(os.getcwd(),f) for f in cytof_files.file_name]
fn

                                file_name    sample_id  time_days  \
0      export_COVID19_994515 d7_Bcell.fcs    994515 d7          0   
1      export_COVID19_994516 d7_Bcell.fcs    994516 d7          0   
2      export_COVID19_994518 d7_Bcell.fcs    994518 d7          0   
3      export_COVID19_994519 d7_Bcell.fcs    994519 d7          0   
4     export_COVID19_994519_d21_Bcell.fcs   994519_d21         21   
..                                    ...          ...        ...   
209  export_COVID19_SCD2020-002_Bcell.fcs  SCD2020_002          0   
210  export_COVID19_SCD2020-003_Bcell.fcs  SCD2020_003          0   
211  export_COVID19_SRD2020_005_Bcell.fcs  SRD2020_005          0   
212  export_COVID19_SRD2020_011_Bcell.fcs  SRD2020_011          0   
213  export_COVID19_SRD2020-009_Bcell.fcs  SRD2020_009          0   

      patient_id condition dataset  y  
0         994515     COVID   train  1  
1         994516     COVID   train  1  
2         994518     COVID   train  1  
3         9

['/home/ewillie/Documents/PhD/DeepLearning_CyTOF/COVID_19_PBMC_Mathew_2020/data/data_sub_10k/export_COVID19_994515 d7_Bcell.fcs',
 '/home/ewillie/Documents/PhD/DeepLearning_CyTOF/COVID_19_PBMC_Mathew_2020/data/data_sub_10k/export_COVID19_994516 d7_Bcell.fcs',
 '/home/ewillie/Documents/PhD/DeepLearning_CyTOF/COVID_19_PBMC_Mathew_2020/data/data_sub_10k/export_COVID19_994518 d7_Bcell.fcs',
 '/home/ewillie/Documents/PhD/DeepLearning_CyTOF/COVID_19_PBMC_Mathew_2020/data/data_sub_10k/export_COVID19_994519 d7_Bcell.fcs',
 '/home/ewillie/Documents/PhD/DeepLearning_CyTOF/COVID_19_PBMC_Mathew_2020/data/data_sub_10k/export_COVID19_994519_d21_Bcell.fcs',
 '/home/ewillie/Documents/PhD/DeepLearning_CyTOF/COVID_19_PBMC_Mathew_2020/data/data_sub_10k/export_COVID19_994520 d7_Bcell.fcs',
 '/home/ewillie/Documents/PhD/DeepLearning_CyTOF/COVID_19_PBMC_Mathew_2020/data/data_sub_10k/export_COVID19_994523 d7_Bcell.fcs',
 '/home/ewillie/Documents/PhD/DeepLearning_CyTOF/COVID_19_PBMC_Mathew_2020/data/data_sub_

In [17]:
##### read fcs file using the flowCore R package #####
# flowCore is a very well maintained R package for reading and analyzing fcs files
# Many of the fcs file related packages in python are a little buggy to use
# Therefore, it is worth the trouble to read the fcs files using R 

r = rp.robjects.r
expr_list = []
for i in range(0,len(fn)):
    fn_i = fn[i]
    r_code = ("library(flowCore);"+
          "library(MetaCyto);"+
          "fn = '"+ fn_i+ "'; "+
          "fcs = read.FCS(fn,truncate_max_range = FALSE);"+
          "expr = fcs@exprs;"+
          "markers = markerFinder(fcs);"+
          "colnames(expr) = markers;"+
          "expr = as.data.frame(expr);"+
         # subsample 10,000 cells +
             "return(as.data.frame(expr));")
    expr =  r(r_code)
    expr_list.append(expr)

R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  library ‘/usr/local/lib/R/site-library’ contains no packages



In [18]:
len(expr_list)

214

In [19]:
markers = []
for i in range(len(expr_list)):
#     expr_list[i].colna = expr_list[i].columns.str.replace(' ', '-')
    markers.extend(expr_list[i].colnames)

markers = Counter(markers)
markers = [k for k, c in markers.items() if c == 214]
print(markers)

for i in range(0,len(expr_list)):
    t1 = expr_list[i] 
    with localconverter(ro.default_converter + pandas2ri.converter):
        t1 = ro.conversion.rpy2py(t1)
    expr_list[i] = t1.loc[:,markers]

['CD45RA.BUV395', 'PD.1.BV421', 'CXCR5.BB515', 'TCF.1.AF647', 'CD38.BUV661', 'CD39.APCFIRE750', 'CD95.BV650', 'EOMES.PEEF610', 'CCR7.BB700', 'KI.67.AF700', 'CD16.BUV615', 'CD27.BUV737', 'CX3CR1.BB790', 'CD20.BUV805', 'T.BET.PE.CY7', 'HLA.DR.BV786']


In [87]:
colsToDrop = ['FSC-A', 'FSC-H', 'FSC-W', 'SSC-A', 'SSC-H', 'SSC-W', 'TIME']

In [20]:
##### transform and format into numpy array
def arcsinh(x):
    return(np.arcsinh(x/5))

coln = expr_list[0].columns
for i in range(len(expr_list)):
    t1 = expr_list[i]
    t1 = t1.apply(arcsinh)
    t1 = t1.values
    shape1 = list(t1.shape)+[1]
    t1 = t1.reshape(shape1)
    expr_list[i] = t1
    
expr_list = np.stack(expr_list)
print("The dimenstion of the data is: ", expr_list.shape)

The dimenstion of the data is:  (214, 10000, 16, 1)


In [21]:
coln

Index(['CD45RA.BUV395', 'PD.1.BV421', 'CXCR5.BB515', 'TCF.1.AF647',
       'CD38.BUV661', 'CD39.APCFIRE750', 'CD95.BV650', 'EOMES.PEEF610',
       'CCR7.BB700', 'KI.67.AF700', 'CD16.BUV615', 'CD27.BUV737',
       'CX3CR1.BB790', 'CD20.BUV805', 'T.BET.PE.CY7', 'HLA.DR.BV786'],
      dtype='object')

In [22]:
allData = {"cytof_files":cytof_files, 
            "expr_list" : expr_list,
            "marker_names" : coln}

with open("../../DL_CyTOF/covid_bcell_allData.obj", "wb") as f:
    pickle.dump(allData, f)